In [1]:
import pandas as pd
import numpy as np
import re
import openpyxl
import nltk
import re
import ast

# mengubah data transaksi list menjadi matriks biner (0 dan 1) agar bisa dibaca mesin
from mlxtend.preprocessing import TransactionEncoder

# # fpgrowth: algoritma pencari pola item; association_rules: Pembuat aturan "Jika-Maka"
from mlxtend.frequent_patterns import fpgrowth, association_rules

from nltk.tokenize import word_tokenize
from nltk.util import bigrams, ngrams

In [2]:
df_raw = pd.read_excel("./data/Data-Sintetis_Pengeluaran-Harian-Mahasiswa.xlsx")
df_raw.head(5)

,Tanggal,Jenis Pengeluaran,Label,Kategori,Nominal (IDR)
0,01/03/2025,Top Up E-Wallet,Hiburan Digital,Keinginan,25000
1,01/03/2025,Print Tugas,Pendidikan,Kebutuhan,17000
2,01/03/2025,Nasi Padang,Makanan,Kebutuhan,28000
3,02/03/2025,Sunscreen,Skincare,Keinginan,85000
4,02/03/2025,Hangout di Mall,Nongkrong,Keinginan,132000


# Filter Daset Keinginan

In [3]:
df_desireCategory = df_raw[df_raw['Kategori'] == "Keinginan"].copy()
df_desireCategory.head(5)

,Tanggal,Jenis Pengeluaran,Label,Kategori,Nominal (IDR)
0,01/03/2025,Top Up E-Wallet,Hiburan Digital,Keinginan,25000
3,02/03/2025,Sunscreen,Skincare,Keinginan,85000
4,02/03/2025,Hangout di Mall,Nongkrong,Keinginan,132000
5,02/03/2025,Moisturizer,Skincare,Keinginan,31000
7,03/03/2025,Kentang Goreng,Makanan,Keinginan,21000


# Preprocessing Data

## Data Cleaning

### Cek Missing Values

In [4]:
missing_values = df_desireCategory.isnull().sum()
print(missing_values)

Tanggal              0
Jenis Pengeluaran    0
Label                0
Kategori             0
Nominal (IDR)        0
dtype: int64


In [5]:
if missing_values.sum() > 0:
  df_missingValueClean = df_desireCategory.dropna()
  print(f"Data berhasil dibersihkan. {len(df_desireCategory) - len(df_missingValueClean)} baris dihapus.")

else:
  df_missingValueClean = df_desireCategory.copy()
  print("Tidak ada missing values")

Tidak ada missing values


In [6]:
df_missingValueClean.head(5)

,Tanggal,Jenis Pengeluaran,Label,Kategori,Nominal (IDR)
0,01/03/2025,Top Up E-Wallet,Hiburan Digital,Keinginan,25000
3,02/03/2025,Sunscreen,Skincare,Keinginan,85000
4,02/03/2025,Hangout di Mall,Nongkrong,Keinginan,132000
5,02/03/2025,Moisturizer,Skincare,Keinginan,31000
7,03/03/2025,Kentang Goreng,Makanan,Keinginan,21000


### Symbol Removal

In [7]:
def removal_symbols(text):

  if pd.isna(text):
    return text

  cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  return cleaned_text

In [8]:
df_removalSymbolsClean = df_missingValueClean.copy()

In [9]:
df_removalSymbolsClean["Jenis Pengeluaran"] = df_removalSymbolsClean["Jenis Pengeluaran"].apply(removal_symbols)

In [10]:
df_removalSymbolsClean.head(5)

,Tanggal,Jenis Pengeluaran,Label,Kategori,Nominal (IDR)
0,01/03/2025,Top Up EWallet,Hiburan Digital,Keinginan,25000
3,02/03/2025,Sunscreen,Skincare,Keinginan,85000
4,02/03/2025,Hangout di Mall,Nongkrong,Keinginan,132000
5,02/03/2025,Moisturizer,Skincare,Keinginan,31000
7,03/03/2025,Kentang Goreng,Makanan,Keinginan,21000


### Standarisasi Teks (lowercase)

In [11]:
def textStandardization(df, targetColumn):
  
  """
    - mengubah teks menjadi huruf kecil
    - menghapus spasi di awal dan akhir teks
  """

  df_clean = df.copy()

  for column in targetColumn:
    if column in df_clean.columns:
      df_clean[column] = df_clean[column].str.lower().str.strip()
    
    else:
      print(f"Kolom '{column}' tidak ditemukan dalam DataFrame.")
      
  return df_clean

In [12]:
df_textStandardization = df_removalSymbolsClean.copy()

In [13]:
target_cols = ["Jenis Pengeluaran", "Label", "Kategori"]

In [14]:
df_textStandardization = textStandardization(df_textStandardization, target_cols)

In [15]:
df_textStandardization.head(10)

,Tanggal,Jenis Pengeluaran,Label,Kategori,Nominal (IDR)
0,01/03/2025,top up ewallet,hiburan digital,keinginan,25000
3,02/03/2025,sunscreen,skincare,keinginan,85000
4,02/03/2025,hangout di mall,nongkrong,keinginan,132000
5,02/03/2025,moisturizer,skincare,keinginan,31000
7,03/03/2025,kentang goreng,makanan,keinginan,21000
10,03/03/2025,masker wajah,skincare,keinginan,29000
11,04/03/2025,top up ewallet,hiburan digital,keinginan,74000
12,04/03/2025,top up game pubg,hiburan digital,keinginan,95000
14,04/03/2025,jus buah,minuman,keinginan,20000
15,04/03/2025,burger,makanan,keinginan,36000


## Metadata Extraction 
-----

### Goals
- Membuat referensi/kamus yang menghubungkan Label dengan Jenis Pengeluaran

- Agar nanti saat hasil FP-Growth menunjukkan "makanan", kita bisa tahu apa aja Jenis Pengeluaran makanan

In [16]:
def extractionMetadata(
    df, 
    groupColumn, 
    detailColumn, 
    save_to_csv=True, 
    file_name="./data/extracted_metadata.csv"):
  
  """
    - mengelompokkan data berdasarkan kolom groupColumn
    - mengekstrak detail unik dari kolom detailColumn
    - menyimpan metadata ke file CSV jika save_to_csv=True
  """

  # grouping and set unik
  try:
    # mengambil data unik dalam bentuk list
    metadataDictionary = df.groupby(groupColumn)[detailColumn].apply(lambda x: list(set(x))).to_dict()
    
  except KeyError as e:
    print(f"error kolom: {e} tidak ditemukan dalam DataFrame.")
    return {}, pd.DataFrame()
  
  # membuat dataframe ringkasan
  df_metadataSummary = pd.DataFrame([
    {
      groupColumn: label,
      detailColumn: str(sorted(items)),
      "Jumlah Unik": len(items)
    }
    for label, items in metadataDictionary.items()
  ])
  
  # menyimpan ke CSV
  if save_to_csv:
    df_metadataSummary.to_csv(file_name, index = False, sep=';', encoding = "utf-8")

  return metadataDictionary, df_metadataSummary

In [17]:
data_dictionary, table_metadataSummary = extractionMetadata(
    df = df_textStandardization,
    groupColumn = "Label",
    detailColumn = "Jenis Pengeluaran",
    save_to_csv = True,
    file_name = "./data/extracted_metadata.csv",
)

In [18]:
table_metadataSummary.head(5)

,Label,Jenis Pengeluaran,Jumlah Unik
0,hiburan digital,"['beli skin game', 'langganan disney', 'top up...",6
1,makanan,"['batagor', 'burger', 'cilok', 'fried chicken'...",7
2,minuman,"['boba tea', 'es teh kekinian', 'jus buah', 'k...",6
3,nongkrong,"['billiard', 'bowling', 'hangout di mall', 'no...",4
4,skincare,"['lip balm', 'lipstik', 'masker wajah', 'moist...",7


## Transaction Aggregation 
---

### Tujuannya adalah mengelompokkan data berdasarkan tanggal, setiap tanggal dianggap sebagai satu transaksi

In [19]:
def aggregateTransactions(
    df, 
    groupColumn, 
    itemColumn, 
    dataFormat="%d/%m/%Y", 
    save_to_csv=True, 
    fileName="./data/aggregated_transactions.csv"):
  
  """
  Fungsi untuk mengelompokkan item berdasarkan ID transaksi (Tanggal).

  Argumens:
    - df (pandas.DataFrame): DataFrame input yang berisi data transaksi.
    - groupColumn (str): Nama kolom yang berisi ID transaksi (Tanggal).
    - detailColumn (str): Nama kolom yang berisi item yang akan digabungkan.
    - itemColumn (str): Nama kolom baru untuk menyimpan daftar item yang digabungkan.
    - dataFormat (str): Format tanggal dalam kolom groupColumn.
  """

  df_aggregation = df.copy()

  # handling tanggal (parsing ke datetime)
  try:
    # mengecek apakah sudah datetime atau belum
    if not pd.api.types.is_datetime64_any_dtype(df_aggregation[groupColumn]):
      df_aggregation[groupColumn] = pd.to_datetime(df_aggregation[groupColumn], format=dataFormat, errors='coerce')
      
  except KeyError as e:
    print(f"gagal parsing tanggal: {e} tidak ditemukan dalam DataFrame.")

  # mengelompokkan item berdasarkan ID transaksi (Tanggal) dan item-itemnya dijadikan list
  try:
    transactionGrouped = df_aggregation.groupby(groupColumn)[itemColumn].apply(list).reset_index()
    transactionGrouped.columns = [groupColumn, "Items"]

  except KeyError as e:
    print(f"error kolom: {e} tidak ditemukan dalam DataFrame.")
    return [], pd.DataFrame()
  
  # membuat list of list (untuk input algoritma)
  transactionList = transactionGrouped["Items"].tolist()

  # menyimpan ke CSV
  if save_to_csv:
    df_save = transactionGrouped.copy()
    df_save["Items String"] = df_save["Items"].apply(lambda x: ', '.join(x))

    df_save[[groupColumn, "Items String"]].to_csv(fileName, index = False, sep=';', encoding = "utf-8")

  return transactionList, transactionGrouped


In [20]:
datasetTransaction, df_transactionAggregation = aggregateTransactions(
    df = df_textStandardization,
    groupColumn = "Tanggal",
    itemColumn = "Label",
    dataFormat = "%d/%m/%Y",
    save_to_csv = True,
    fileName = "./data/aggregated_transactions.csv"
) 

In [21]:
df_transactionAggregation.head(5)

,Tanggal,Items
0,2025-03-01,[hiburan digital]
1,2025-03-02,"[skincare, nongkrong, skincare]"
2,2025-03-03,"[makanan, skincare]"
3,2025-03-04,"[hiburan digital, hiburan digital, minuman, ma..."
4,2025-03-05,"[skincare, hiburan digital]"


## One-Hot Encoding (Transformasi Ke Matriks Biner)

In [22]:
def encodeTransactionsLabel(
    transactionList, 
    df_reference, 
    dateTimeColumn="Tanggal", 
    save_to_csv=True, 
    fileName="./data/encodedTransactionsLabel.csv"):
  
  """
    mengubah data label Items pada dataset Transaction Aggregations menjadi matriks biner (0 dan 1)

    Argumens:
      - transactionList (list of list): daftar transaksi yang berisi item-itemnya
      - df_reference (pandas.DataFrame): DataFrame referensi untuk mengambil kolom tanggal
      - dateTimeColumn (str): nama kolom tanggal pada df_reference
      - save_to_csv (bool): apakah menyimpan hasil encoding ke file CSV
      - fileName (str): nama file CSV untuk menyimpan hasil encoding
  """

  # inisialisasi & fitting TransactionEncoder
  try:
    transactionEncoder = TransactionEncoder()
    transactionEncoder_ary = transactionEncoder.fit(transactionList).transform(transactionList)

    # membuat DataFrame dari array biner
    df_encodedTransactions = pd.DataFrame(transactionEncoder_ary, columns=transactionEncoder.columns_)

    # konversi boolean (True/False) 
    df_encodedTransactions = df_encodedTransactions.astype(bool)

    # menambahkan kolom tanggal dari df_reference
    if dateTimeColumn in df_reference.columns:
      df_encodedTransactions.insert(0, dateTimeColumn, df_reference[dateTimeColumn].values)

    else:
      print(f"warning: kolom '{dateTimeColumn}' tidak ditemukan dalam df_reference")

  except Exception as e:
    print(f"gagal melakukan encoding transaksi: {e}")
    return pd.DataFrame(), None

  # menyimpan ke CSV
  if save_to_csv:
    df_encodedTransactions.to_csv(
      fileName, 
      index=False, 
      sep=';', 
      encoding="utf-8")

  return df_encodedTransactions, transactionEncoder 

In [23]:
df_oneHotEncoded, encoderObject = encodeTransactionsLabel(
    transactionList=datasetTransaction,
    df_reference=df_transactionAggregation,
    dateTimeColumn="Tanggal",
    save_to_csv=True,
    fileName="./data/encodedTransactionsLabel.csv",
)

In [24]:
df_oneHotEncoded.head(5)

,Tanggal,hiburan digital,makanan,minuman,nongkrong,skincare
0,2025-03-01,True,False,False,False,False
1,2025-03-02,False,False,False,True,True
2,2025-03-03,False,True,False,False,True
3,2025-03-04,True,True,True,False,False
4,2025-03-05,True,False,False,False,True


# Frequent Pattern Mining (Algoritma FP-Growth)

In [47]:
def generateFrequentItemsets(
    df_encoded, 
    min_support=0.1, 
    dateTimeColumn="Tanggal", 
    save_to_csv=True, 
    fileName="./data/frequent_pattern_mining.csv"):
  
  """
    fungsi untuk menghasilkan frequent itemsets menggunakan algoritma FP-Growth

    Argumens:
      - df_encoded (pandas.DataFrame): DataFrame yang berisi data transaksi dalam format one-hot encoding
      - minSupport (float): nilai minimum support untuk frequent itemsets
      - dateTimeColumn (str): nama kolom tanggal pada df_encoded
      - save_to_csv (bool): apakah menyimpan hasil frequent itemsets ke file CSV
      - fileName (str): nama file CSV untuk menyimpan hasil frequent itemsets
  """

  # preprocessing data (cleaning untuk mining)
  if dateTimeColumn in df_encoded.columns:
    df_mining = df_encoded.drop(columns=[dateTimeColumn])

  else:
    df_mining = df_encoded.copy()

  # memastikan data tidak kosong
  if df_mining.empty:
    print(f"error: data untuk mining kosong.")
    return pd.DataFrame()
  
  # menjalankan algoritma fp-growth
  try:
    frequentItemsets = fpgrowth(
      df_mining, 
      min_support=min_support, 
      use_colnames=True)

    if frequentItemsets.empty:
        print(f"peringatan: tidak ditemukan frequent itemsets dengan support {min_support} yang diberikan.")
        return pd.DataFrame()
    
  except Exception as e:
    print(f"gagal menghasilkan frequent itemsets: {e}")
    return pd.DataFrame()
  
  # menambahkan kolom 'length' untuk menghitung jumlah item dalam setiap itemset (enrichment)
  frequentItemsets["length"] = frequentItemsets["itemsets"].apply(lambda x: len(x))

  # menambahkan kolom count (jumlah transaksi absolut)
  totalTransactions = len(df_mining)
  frequentItemsets["count"] = (frequentItemsets["support"] * totalTransactions).astype(int)

  # mengurutkan support secara menurun (descending)
  frequentItemsets = frequentItemsets.sort_values(by="support", ascending=False).reset_index(drop=True)

  # menyimpan ke CSV
  if save_to_csv:
    df_save = frequentItemsets.copy()
    df_save["itemsets"] = df_save["itemsets"].apply(lambda x: ', '.join(list(x)))

    df_save.to_csv(
      fileName,
      index=False,
      sep=';',
      encoding="utf-8"
    )

  return frequentItemsets
  

In [48]:
df_frequentItemsets = generateFrequentItemsets(
    df_encoded=df_oneHotEncoded,
    min_support=0.1,
    dateTimeColumn="Tanggal",
    save_to_csv=True,
    fileName="./data/frequentPatternMining.csv",
)

In [49]:
df_frequentItemsets.head(5)

,support,itemsets,length,count
0,0.428571,(skincare),1,12
1,0.392857,(makanan),1,11
2,0.357143,(hiburan digital),1,10
3,0.214286,(minuman),1,6
4,0.178571,(nongkrong),1,5


# Creating Association Rules (Rule Generation)

In [50]:
def generateAssociationRules(
        df_frequencyItemsets, 
        min_confidence=0.5, 
        metric_type="confidence", 
        save_to_csv=True, 
        file_name="./data/association_rules.csv"):
    
    """
    fungsi untuk menghasilkan aturan asosiasi dari frequent itemsets

    argumens:
        - df_frequencyItemsets (pandas.DataFrame): DataFrame yang berisi frequent itemsets
        - min_confidence (float): nilai minimum confidence untuk aturan asosiasi
        - metric_type (str): jenis metrik untuk mengukur kekuatan aturan (default: "confidence")
        - save_to_csv (bool): apakah menyimpan hasil aturan asosiasi ke file CSV
        - file_name (str): nama file CSV untuk menyimpan hasil aturan asosiasi
    """

    # validasi input
    if df_frequencyItemsets.empty:
        return pd.DataFrame()
    
    # membuat aturan asosiasi
    try:
        associationRules = association_rules(
            df_frequencyItemsets, 
            metric=metric_type, 
            min_threshold=min_confidence)
        
        if associationRules.empty:
            print(f"warning: tidak ditemukan aturan dengan {metric_type} >= {min_confidence}.")

            threshold = [0.4, 0.3, 0.2, 0.1, 0.05]
            
            for confidence in threshold:
                if confidence >= min_confidence: 
                    continue

                associationRules = association_rules(
                    df_frequencyItemsets,
                    metric=metric_type,
                    min_threshold=confidence
                )

                if not associationRules.empty:
                    # update nilai min_confidence untuk laporan
                    min_confidence = confidence
                    break
            
            if associationRules.empty:
                return pd.DataFrame()

    except Exception as e:
        return pd.DataFrame()
    
    # mengurutkan berdasarkan lift secara menurun (kualitas aturan)
    associationRules = associationRules.sort_values("lift", ascending=False).reset_index(drop=True)

    # konversi frozenset ke string agar lebih mudah dibaca
    associationRules["antecedents"] = associationRules["antecedents"].apply(lambda x: ', '.join(list(x)))
    associationRules["consequents"] = associationRules["consequents"].apply(lambda x: ', '.join(list(x)))

    # membuat kolom rules agar lebih mudah dibaca
    associationRules["rules"] = associationRules.apply(lambda x: f"{{{x["antecedents"]}}} ->{{{x["consequents"]}}}", axis=1)

    # menyimpan ke CSV
    if save_to_csv:
        columns_to_save = [
            "rules",
            "antecedents",
            "consequents",
            "support",
            "confidence",
            "lift",
            "leverage",
            "conviction"
        ]

        associationRules[columns_to_save].to_csv(
            file_name,
            index=False,
            sep=';',
            encoding="utf-8"
        )
    
    return associationRules


In [51]:
df_associationRules = generateAssociationRules(
    df_frequencyItemsets=df_frequentItemsets,
    min_confidence=0.5,
    save_to_csv=True,
    file_name="./data/associationRules.csv",
)

In [52]:
df_associationRules.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,rules
0,hiburan digital,skincare,0.357143,0.428571,0.142857,0.400000,0.933333,1.0,-0.010204,0.952381,-0.100000,0.222222,-0.050000,0.366667,{hiburan digital} ->{skincare}
1,skincare,hiburan digital,0.428571,0.357143,0.142857,0.333333,0.933333,1.0,-0.010204,0.964286,-0.111111,0.222222,-0.037037,0.366667,{skincare} ->{hiburan digital}
2,hiburan digital,makanan,0.357143,0.392857,0.107143,0.300000,0.763636,1.0,-0.033163,0.867347,-0.325000,0.166667,-0.152941,0.286364,{hiburan digital} ->{makanan}


# Insight Enrichment (Metadata Mapping)